In [1]:
from hypersurface import *
import sympy as sp
from pprint import pprint
import time

In [2]:
z0, z1, z2, z3, z4= sp.symbols('z0, z1, z2, z3, z4')
Z = [z0,z1,z2,z3,z4]
f = z0**5 + z1**5 + z2**5 + z3**5 + z4**5
HS = Hypersurface(Z, f, 1000)

In [3]:
k = 5
HS.set_k(k)

In [4]:
# for defining S
Ss = []
Js = []
Rs = []
OO_bars = []
for patch in HS.patches:
    for subpatch in patch.patches:
        for point in subpatch.points:
            Ss.append(np.array(subpatch.sections(point)))
            Js.append(subpatch.sections_jacobian(point))
            Rs.append(subpatch.restriction(point))
            OO_bars.append(subpatch.omega_omegabar(point))

In [5]:
S_arr = np.array([[s] for s in Ss])

In [6]:
Jd_arr = np.conj(np.array(Js))

In [7]:
J_arr = np.transpose(np.array(Js),axes = (0,2,1))

In [8]:
Sd_arr = np.transpose(np.conj(S_arr),axes=(0,2,1))

In [9]:
R_arr = np.array(Rs)

In [10]:
def H_vec(h_mat,nn):
    H_arr = []
    for _ in range(nn):
        H_arr.append(h_mat)
    return np.array(H_arr)

In [11]:
H_id = np.identity(HS.n_sections)
H_id_arr = H_vec(H_id,5000)

In [12]:
def Vec_VolForm(h_matrix):
    H_jd = np.matmul(h_matrix,Jd_arr)
    A = np.matmul(J_arr,H_jd)
    b = np.matmul(S_arr,H_jd)
    B = np.matmul(np.transpose(np.conj(b),axes=(0,2,1)),b)
    alpha = np.matmul(np.matmul(S_arr,H_id_arr),Sd_arr)
    G = A/alpha - B/alpha**2
    V_arr = np.matmul(np.transpose(R_arr,axes=(0,2,1)),np.matmul(G,np.conj(R_arr)))
    VolForm_arr = np.linalg.det(V_arr)
    return VolForm_arr

In [13]:
start = time.time()
Vec_VolForm(H_id_arr)
print(time.time()-start)

2.05763578414917
